## `pytraj` vs `mdtraj`.

I just want to test how slow and how fast `pytraj` is to further optimize the speed. 
This comparison won't be fair since I have limited knowdlege about `mdtraj`. And there is different in designing two packages (`cpptraj` used double precision while `mdtraj` use (mostly?) float32)

In [37]:
from pytraj.__version__ import __version__ as p_version
print (p_version)
from mdtraj.version import version as m_version
print (m_version)

0.1.2.dev3
1.4.0.dev0.dev-bb6923f


## loading data

In [48]:
# data is stored in local disk, generated from TIP3P REMD, netcdf

from pytraj import io
import mdtraj as md

top_name = "../tests/data/nogit/remd/myparm.parm7"
filename = "../tests/data/nogit/remd/remd.x.000"

!du -sh "../tests/data/nogit/remd/remd.x.000"
print ()
!head "../tests/data/nogit/remd/myparm.parm7"

200M	../tests/data/nogit/remd/remd.x.000

%VERSION  VERSION_STAMP = V0001.000  DATE = 03/13/14  02:23:52                  
%FLAG TITLE                                                                     
%FORMAT(20a4)                                                                   
default_name                                                                    
%FLAG POINTERS                                                                  
%FORMAT(10I8)                                                                   
   17443      16   17165     287     603     386    1257    1129       0       0
   25521    5666     287     386    1129      61     139     179      32       1
       0       0       0       0       0       0       0       2      24       0
       0


In [39]:
# load whole traj into memory by FrameArray in pytraj
# I don't use %timeit because it's terriblly slow in my computer

%time io.load(filename, top_name)[:]
# using cpptraj's class (Trajin_Single)
%time io.load(filename, top_name) # the data is actually still in disk :D

%time md.load_netcdf(filename, top=top_name)

# pytraj fa: -1 (but not that slow)

CPU times: user 2.11 s, sys: 466 ms, total: 2.58 s
Wall time: 2.58 s
CPU times: user 213 ms, sys: 17 ms, total: 230 ms
Wall time: 230 ms
CPU times: user 1.87 s, sys: 335 ms, total: 2.21 s
Wall time: 2.41 s


<mdtraj.Trajectory with 1000 frames, 17443 atoms, 5666 residues, and unitcells at 0x2aab0405c588>

## calc_rmsd

In [40]:
f0 = fa[0]
%timeit [frame.rmsd(f0) for frame in fa]
%timeit md.rmsd(m_traj, m_traj, 0)

# mdtraj has very fast rmsd calculation (about 10 times faster). they use "openmp" (?) and float32 while we used float64

10 loops, best of 3: 99.5 ms per loop
100 loops, best of 3: 8.74 ms per loop


## calc_radgyr

In [41]:
%timeit fa.calc_radgyr()
%timeit md.compute_rg(m_traj)

# pytraj/cpptraj is about 5 times faster

10 loops, best of 3: 53.2 ms per loop
1 loops, best of 3: 267 ms per loop


## calc_dssp

In [42]:
## calc_dssp
import numpy as np
%timeit fa.calc_dssp(dtype='ndarray')

10 loops, best of 3: 44.1 ms per loop


In [43]:
%timeit md.compute_dssp(m_traj)

1 loops, best of 3: 15.5 s per loop


## calc_COM

In [44]:
%timeit fa.calc_COM()
%timeit md.compute_center_of_mass(m_traj)

# almost the same

10 loops, best of 3: 58.9 ms per loop
10 loops, best of 3: 69.6 ms per loop


## calc_distance

In [45]:
%timeit fa.calc_distance("@1 @300")

indices = np.array([[0, 299],])
%timeit md.compute_distances(m_traj, indices)

# mdtraj is much faster for single calculation (62 times faster). Not sure about including mask like :2-100@CB,CA ...

10 loops, best of 3: 27.7 ms per loop
1000 loops, best of 3: 408 µs per loop


## calc_psi

In [46]:
%timeit fa.calc_multidihedral("psi")
%timeit md.compute_psi(m_traj)

# mdtraj is about 10 times faster. Not sure about mask selection

1 loops, best of 3: 493 ms per loop
10 loops, best of 3: 62.5 ms per loop


## calc_phi

In [47]:
%timeit fa.calc_multidihedral("phi")
%timeit md.compute_phi(m_traj)

# same result as calc_psi

1 loops, best of 3: 500 ms per loop
10 loops, best of 3: 63.1 ms per loop


In [55]:
# search all dihedral?

%timeit fa.calc_multidihedral() # search all in pytraj/cpptraj

%timeit md.compute_chi1(m_traj)
%timeit md.compute_chi2(m_traj)
%timeit md.compute_chi3(m_traj)
%timeit md.compute_chi4(m_traj) 
%timeit md.compute_phi(m_traj)
%timeit md.compute_psi(m_traj)

1 loops, best of 3: 1.48 s per loop
1 loops, best of 3: 284 ms per loop
1 loops, best of 3: 342 ms per loop
1 loops, best of 3: 228 ms per loop
10 loops, best of 3: 115 ms per loop
10 loops, best of 3: 58.5 ms per loop
10 loops, best of 3: 57.9 ms per loop


## saving files

In [58]:
# netcdf

%timeit fa.save("fa.nc", overwrite=True)
%timeit m_traj.save("m_traj.nc")

1 loops, best of 3: 205 ms per loop
10 loops, best of 3: 118 ms per loop


In [59]:
# dcd

%timeit fa.save("fa.dcd", overwrite=True)
%timeit m_traj.save("m_traj.dcd")

10 loops, best of 3: 61.1 ms per loop
10 loops, best of 3: 78.5 ms per loop


In [60]:
# binpos

%timeit fa.save("fa.binpos", overwrite=True)
%timeit m_traj.save("m_traj.binpos")

10 loops, best of 3: 63 ms per loop
1 loops, best of 3: 336 ms per loop


In [62]:
# xtc # not sure cpptraj supports

%timeit fa.save("fa.xtc", overwrite=True)
%timeit m_traj.save("m_traj.xtc")

1 loops, best of 3: 4.01 s per loop
1 loops, best of 3: 196 ms per loop
